In [1]:
from fastai.text.all import *
import fastai

In [2]:
fastai.__version__

'2.0.19'

In [4]:
path = untar_data(URLs.IMDB)
path.ls()

(#8) [Path('/home/advaitmb/.fastai/data/imdb/imdb.vocab'),Path('/home/advaitmb/.fastai/data/imdb/tmp_clas'),Path('/home/advaitmb/.fastai/data/imdb/test'),Path('/home/advaitmb/.fastai/data/imdb/train'),Path('/home/advaitmb/.fastai/data/imdb/unsup'),Path('/home/advaitmb/.fastai/data/imdb/models'),Path('/home/advaitmb/.fastai/data/imdb/tmp_lm'),Path('/home/advaitmb/.fastai/data/imdb/README')]

In [2]:
path = Path('/home/advaitmb/notebooks/projects/PreDiction/nbs')

In [5]:
dls_lm = TextDataLoaders.from_folder(path, is_lm=True, valid_pct=0.1)

In [6]:
dls_lm.show_batch()

,text,text_
0,"xxbos i had a different experience with this movie - it never got charming , or delightful , or funny for me . one big clue that this was not your typical movie was that the label gave no indication of the xxmaj xxunk ) spoken in the film . another was the lack of choices re subtitles . \n\n i found the lack of dialogue annoying , especially when accompanied by","i had a different experience with this movie - it never got charming , or delightful , or funny for me . one big clue that this was not your typical movie was that the label gave no indication of the xxmaj xxunk ) spoken in the film . another was the lack of choices re subtitles . \n\n i found the lack of dialogue annoying , especially when accompanied by exaggerated"
1,"the line into soppy sentimentality . xxmaj naturally sweet - no added sugar . \n\n xxmaj one small point : it seems to me that the mild objections raised about xxmaj ustinov 's character xxmaj pendelton being able to walk in and defeat the system security ignore the fact that xxmaj pendelton is clearly a genius / savant at this sort of thing . xxmaj yes , the film was pretty computer","line into soppy sentimentality . xxmaj naturally sweet - no added sugar . \n\n xxmaj one small point : it seems to me that the mild objections raised about xxmaj ustinov 's character xxmaj pendelton being able to walk in and defeat the system security ignore the fact that xxmaj pendelton is clearly a genius / savant at this sort of thing . xxmaj yes , the film was pretty computer illiterate"
2,"does follow the same spirit as all the other xxmaj troma movies [ except xxmaj combat xxmaj shock … that sucker was depressing ] but it 's not only for xxmaj troma fans . xxmaj anybody who appreciates lowbrow visuals and a hilarious script will without a doubt fall in love with this movie . \n\n xxmaj do n't expect pretentious , artistic - wannabe crap like the version of xxup xxunk","follow the same spirit as all the other xxmaj troma movies [ except xxmaj combat xxmaj shock … that sucker was depressing ] but it 's not only for xxmaj troma fans . xxmaj anybody who appreciates lowbrow visuals and a hilarious script will without a doubt fall in love with this movie . \n\n xxmaj do n't expect pretentious , artistic - wannabe crap like the version of xxup xxunk with"
3,"but no music , and the silly xxmaj american influenced "" kings "" who have music but no girls . xxmaj the "" ins "" show ingenuity in xxunk the situation . \n\n xxmaj absa uses local residents of xxmaj popenguine , xxmaj senegal for his movie , not actors . xxmaj it is filmed on location by xxmaj africans . xxmaj this works well to provide an authentic setting and to","no music , and the silly xxmaj american influenced "" kings "" who have music but no girls . xxmaj the "" ins "" show ingenuity in xxunk the situation . \n\n xxmaj absa uses local residents of xxmaj popenguine , xxmaj senegal for his movie , not actors . xxmaj it is filmed on location by xxmaj africans . xxmaj this works well to provide an authentic setting and to show"
4,", and we still "" crack - up "" when we think about it xxrep 3 ! xxmaj the expressions on the faces , the antics , the characters , the situations , are truly inspired . i was not aware , although not surprised , that there was a whole series of episodes "" spinning - off "" from this original , and i would n't mind seeing those . xxmaj","and we still "" crack - up "" when we think about it xxrep 3 ! xxmaj the expressions on the faces , the antics , the characters , the situations , are truly inspired . i was not aware , although not surprised , that there was a whole series of episodes "" spinning - off "" from this original , and i would n't mind seeing those . xxmaj sometimes"
5,"jason xxmaj mewes to play xxmaj jay . xxmaj universal did n't want jay to play jay , they wanted xxmaj seth xxmaj green or xxmaj breckin xxmaj meyer , but even they realized that xxmaj jay should be xxmaj jay , xxmaj jay is based on the wa

In [7]:
learn_lm = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [48]:
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

1
0
Tesla K80


In [24]:
learn_lm.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.550113,4.124372,0.285586,61.828968,2:09:19


In [26]:
learn_lm.predict("This is ", n_words=20)

'This is a long - run , wherein Mr . Sondheim can perform a seemingly contradictory , difficult movie .'

In [3]:
learn_lm = load_learner('/home/advaitmb/notebooks/projects/PreDiction/api/5epochs_imdb_lm.pkl')

In [4]:
learn = load_learner('/home/advaitmb/notebooks/projects/PreDiction/api/imdb_sentiment_classifier.pkl')

In [20]:
learn_lm.path = Path('/home/advaitmb/notebooks/projects/PreDiction/nbs/')

In [17]:
learn_lm.save('/home/advaitmb/notebooks/projects/PreDiction/nbs/test')

Path('/home/advaitmb/notebooks/projects/PreDiction/nbs/test.pth')

In [5]:
learn_lm.predict("some people believe", n_words=40, temperature=0.7)

"some people believe this movie is a comedy , and it is not . It 's about a man who has no idea what he 's doing and who is involved in it , to his wife , his wife , and"

In [21]:
learn_lm.save_encoder('finetuned')

In [22]:
dls_clas = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', text_vocab=dls_lm.vocab)

In [23]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [24]:
learn = learn.load_encoder('finetuned')

In [25]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.579591,0.377221,0.832360,07:28


In [26]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.369226,0.269467,0.888720,08:19


In [27]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.273259,0.213489,0.916640,11:56


In [28]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.223270,0.205274,0.919680,14:58
1,0.211528,0.202441,0.923600,15:01


In [29]:
learn.export('/home/advaitmb/notebooks/projects/PreDiction/api/imdb_sentiment_classifier.pkl')

In [8]:
learn.predict??

In [60]:
torch.Tensor([torch.Tensor([0.234]), torch.Tensor([0.214]), torch.Tensor([0.3334]), torch.Tensor([0.231])])[[0,1]]

tensor([0.2340, 0.2140])

In [7]:
def beam_search(learn, clf, bias, text:str, n_words:int, no_unk:bool=True, top_k:int=10, beam_sz:int=100, temperature:float=1.,
                    sep:str=' ', decoder=decode_spec_tokens):
    learn.model.reset()
    learn.model.eval()
    learn.model.cuda()
    clf.model.cuda()
    idx = learn.dls.test_dl([text]).items[0][None].cuda()
    nodes = None
    nodes = idx.clone().cuda()
    scores = idx.new_zeros(1).float()
    if no_unk: unk_idx = learn.dls.vocab.index(UNK)
    with torch.no_grad():
        for k in progress_bar(range(n_words), leave=False):
            out = F.log_softmax(learn.model(idx)[0][:,-1], dim=-1)
            if no_unk: out[:, unk_idx] = -float('Inf')
            values, indices = out.topk(top_k, dim=-1)
            scores = (-values + scores[:,None]).view(-1)
            indices_idx = torch.arange(0,nodes.size(0))[:,None].expand(nodes.size(0), top_k).contiguous().view(-1)
            sort_idx = scores.argsort()[:beam_sz]
            scores = scores[sort_idx]
            nodes = torch.cat([nodes[:,None].expand(nodes.size(0),top_k,nodes.size(1)),
                                    indices[:,:,None].expand(nodes.size(0),top_k,1),], dim=2)
            nodes = nodes.view(-1, nodes.size(2))[sort_idx]
            learn.hidden = [(h[0][:,indices_idx[sort_idx],:],h[1][:,indices_idx[sort_idx],:]) for h in learn.model[0].hidden]
            idx = nodes[:,-1][:,None]
        if temperature != 1.: scores.div_(temperature)
        
        num = learn.dls.train_ds.numericalize
        sep = learn.dls.train_ds.tokenizer.sep
        bias_idx = []
        for idx, node in enumerate(nodes):
            tokens = [num.vocab[i] for i in node if num.vocab[i] not in [BOS, PAD]]
            phrase = sep.join(decoder(tokens))
            if clf.predict(phrase)[0] == bias:
                bias_idx.append(idx)
        print(bias_idx)
        node_idx = torch.multinomial(torch.exp(-scores[bias_idx]), 1).item()
        print(node_idx)
        tokens = [num.vocab[i] for i in nodes[node_idx][1:] if num.vocab[i] not in [BOS, PAD]]
        sep = learn.dls.train_ds.tokenizer.sep
        return sep.join(decoder(tokens))

In [8]:
beam_search(learn=learn_lm, clf=learn, bias='pos', text='the', n_words=15)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
73


'the movie . The All - i - love is Gun Great'

In [15]:
learn_lm.predict?

In [32]:
learn.predict??

In [43]:
d = {0:'neg', 1:'pos'}
phrase = ["this is a good movie", "this is amazing", "this is bad", "this is terrible"]
dl = learn.dls.test_dl(phrase, rm_type_tfms=None, num_workers=0)
print(idx)
inp,preds,_,dec_preds = learn.get_preds(dl=dl, with_input=True, with_decoded=True)
print(dec_preds)
# print(cl)
# if clf.predict(phrase)[0] == bias:
#     bias_idx.append(idx)

this is amazing


tensor([1, 1, 0, 0])


In [45]:
for p in dec_preds:
    print(p.item())

1
1
0
0


In [19]:
learn.predict("hello there no")

('neg', tensor(0), tensor([0.6538, 0.3462]))

In [35]:
a = ["this is a good movie", "this is amazing", "this is bad", "this is terrible"]
learn.predict(a[2])

('neg', tensor(0), tensor([9.9960e-01, 3.9635e-04]))

In [53]:
learn.model.to('cpu')
learn_lm.model.to('cpu')

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(60008, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(60008, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=60008, bias=True)
    (output_dp): RNNDropout()
  )
)

In [16]:
learn_lm.model.cuda()
learn.model.cuda()

SequentialRNN(
  (0): SentenceEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(60008, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(60008, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): Sequential(
      (0): LinBnDrop(
        (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Dropout(p=0.2, inplace=False)
        (2): Linear(in_features=1200, out_features=50, bias=False)
        (3): ReLU(inplace=True

In [ ]:
learn_lm.model.

In [17]:
def beam_search_modified_with_clf(learn, clf, bias, text:str, confidence:float, no_unk:bool=True, no_punct:bool=True, top_k:int=10, beam_sz:int=10, temperature:float=1.,
                    sep:str=' ', decoder=decode_spec_tokens):
    bias_map = {0:'neg', 1:'pos'}
    learn.model.reset()
    learn.model.eval()

    idx = learn.dls.test_dl([text]).items[0][None].cuda()
    nodes = None
    nodes = idx.clone()
    scores = idx.new_zeros(1).float()
    if no_unk: unk_idx = learn.dls.vocab.index(UNK)
    

    with torch.no_grad():
        while(torch.exp(-scores[0])>confidence):
            out = F.log_softmax(learn.model(idx)[0][:,-1], dim=-1)
            
            if no_unk: out[:, unk_idx] = -float('Inf')

            values, indices = out.topk(top_k, dim=-1)
            
            scores = (-values + scores[:,None]).view(-1)
            indices_idx = torch.arange(0,nodes.size(0))[:,None].expand(nodes.size(0), top_k).contiguous().view(-1)
            sort_idx = scores.argsort()[:beam_sz]
            scores = scores[sort_idx]

            nodes = torch.cat([nodes[:,None].expand(nodes.size(0),top_k,nodes.size(1)),
                                    indices[:,:,None].expand(nodes.size(0),top_k,1),], dim=2)
            nodes = nodes.view(-1, nodes.size(2))[sort_idx]            
            learn.hidden = [(h[0][:,indices_idx[sort_idx],:],h[1][:,indices_idx[sort_idx],:]) for h in learn.model[0].hidden]
            idx = nodes[:,-1][:,None]
        
        
        num = learn.dls.train_ds.numericalize
        sep = learn.dls.train_ds.tokenizer.sep
        bias_idx = []
        phrases = []
        
        for idx, node in enumerate(nodes):
            tokens = [num.vocab[i] for i in node if num.vocab[i] not in [BOS, PAD]]
            phrase = sep.join(decoder(tokens))
            phrases.append(phrase)
            
        dl = clf.dls.test_dl(phrases, rm_type_tfms=None, num_workers=0)
        inp,preds,_,dec_preds = clf.get_preds(dl=dl, with_input=True, with_decoded=True)
        
        for p in dec_preds:
            if bias_map[p.item()] == bias:
                bias_idx.append(idx)


        if temperature != 1.: scores.div_(temperature)
        node_idx = torch.multinomial(torch.exp(-scores[bias_idx]), 1).item()
        tokens = [num.vocab[i] for i in nodes[node_idx][1:] if num.vocab[i] not in [BOS, PAD]]
        return sep.join(decoder(tokens))

In [18]:
beam_search_modified_with_clf(learn=learn_lm, clf=learn, bias='neg', text='this is', confidence=0.01, temperature=0.7)

'this is one of the most'

In [93]:
def beam_search_modified_with_clf(learn, clf, bias, text:str, confidence:float, no_unk:bool=True, no_punct:bool=True, top_k:int=10, beam_sz:int=100, temperature:float=1.,
                    sep:str=' ', decoder=decode_spec_tokens):
    bias_map = {0:'neg', 1:'pos'}
    learn.model.reset()
    learn.model.eval()

    idx = learn.dls.test_dl([text]).items[0][None]
    nodes = None
    nodes = idx.clone()
    scores = idx.new_zeros(1).float()
    if no_unk: unk_idx = learn.dls.vocab.index(UNK)
    

    with torch.no_grad():
        while(torch.exp(-scores[0])>confidence):
            out = F.log_softmax(learn.model(idx)[0][:,-1], dim=-1)
            
            if no_unk: out[:, unk_idx] = -float('Inf')

            values, indices = out.topk(top_k, dim=-1)
            
            scores = (-values + scores[:,None]).view(-1)
            indices_idx = torch.arange(0,nodes.size(0))[:,None].expand(nodes.size(0), top_k).contiguous().view(-1)
            sort_idx = scores.argsort()[:beam_sz]
            scores = scores[sort_idx]

            nodes = torch.cat([nodes[:,None].expand(nodes.size(0),top_k,nodes.size(1)),
                                    indices[:,:,None].expand(nodes.size(0),top_k,1),], dim=2)
            nodes = nodes.view(-1, nodes.size(2))[sort_idx]            
            learn.hidden = [(h[0][:,indices_idx[sort_idx],:],h[1][:,indices_idx[sort_idx],:]) for h in learn.model[0].hidden]
            idx = nodes[:,-1][:,None]
        
        
        num = learn.dls.train_ds.numericalize
        sep = learn.dls.train_ds.tokenizer.sep

        if temperature != 1.: scores.div_(temperature)
        node_idx = torch.multinomial(torch.exp(-scores), 20)
#         node_idx = torch.exp(-scores).topk(20, dim=-1).indices
        print(node_idx)
        phrases = []
        biased_idx = []
        for idx in node_idx:
            tokens = [num.vocab[i] for i in nodes[idx] if num.vocab[i] not in [BOS, PAD]]
            phrase = sep.join(decoder(tokens))
            phrases.append(phrase)
            biased_idx.append(idx)
            
        dl = clf.dls.test_dl(phrases, rm_type_tfms=None, num_workers=0)
        inp,preds,_,dec_preds = clf.get_preds(dl=dl, with_input=True, with_decoded=True)
        
        for i in range(len(inp)):
            if bias_map[dec_preds[i].item()] == bias:
                tokens = [num.vocab[i] for i in inp[i][1:] if num.vocab[i] not in [BOS, PAD]]
                return sep.join(decoder(tokens))
                
             
        
    return ""

In [97]:
beam_search_modified_with_clf(learn=learn_lm, clf=learn, bias='neg', text='this is', confidence=0.001, temperature=0.8)

tensor([60,  0,  7, 41, 51,  1, 95, 29, 15,  2, 27,  5,  4, 85, 53, 45, 30, 26,
        78,  8])


'this is one of the worst movies i have ever'

In [13]:
torch.multinomial??

In [55]:
a = ['this is something i was the', 'this is something of a low', 'this is something of a waste', 'this is something of a film', 'this is something of a movie', 'this is something i have -', 'this is something of a bad', 'this is something that is', 'this is something of a great', 'this is something i was a', 'this is something of a classic', 'this is something i am i', 'this is something of a horror', "this is something you ca n't", 'this is something you can be', 'this is something about the', "this is something i would n't", 'this is something of a', 'this is something that would be', ]

In [57]:
for ent in check:
    print(ent, learn.predict('this is something i am not'))

this is something of the ('neg', tensor(0), tensor([0.5347, 0.4653]))


this is something of a low ('neg', tensor(0), tensor([0.5347, 0.4653]))


this is something that 's not ('neg', tensor(0), tensor([0.5347, 0.4653]))


this is something of a bad ('neg', tensor(0), tensor([0.5347, 0.4653]))


this is something i would ('neg', tensor(0), tensor([0.5347, 0.4653]))


...

this is something i do n't ('neg', tensor(0), tensor([0.5347, 0.4653]))


this is something of a great ('neg', tensor(0), tensor([0.5347, 0.4653]))


this is something i was the ('neg', tensor(0), tensor([0.5347, 0.4653]))


this is something i think it ('neg', tensor(0), tensor([0.5347, 0.4653]))


this is something about the ('neg', tensor(0), tensor([0.5347, 0.4653]))


In [3]:
import asyncio
# def f(x):
#     return x*x

# if __name__ == '__main__':
#     pool = Pool(processes=1)              # Start a worker processes.
#     result = pool.apply_async(f, [10], callback) # Evaluate "f(10)" asynchronously calling callback when finished.


In [2]:
from fastai.text.all import *

In [7]:
lstm_model = load_learner('../api/5epochs_imdb_lm.pkl')
lstm_model.model.to('cuda')

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(60008, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(60008, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=60008, bias=True)
    (output_dp): RNNDropout()
  )
)

In [11]:
async def complete_word(language_model, text, final_word, no_unk=True, decoder=decode_spec_tokens, temperature=1.):
    print("inside complete word loop", sys.stdout)
    language_model.model.reset()
    language_model.model.eval()
    numericalize = language_model.dls.train_ds.numericalize
    idxs = idxs_all = language_model.dls.test_dl([text]).items[0].to(language_model.dls.device)
    if no_unk: unk_idx = language_model.dls.vocab.index(UNK)
        
    with torch.no_grad():
        try:
            with language_model.no_bar(): preds,_ = language_model.get_preds(dl=[(idxs[None],)])
        except:
            print("exiting complete word loop with error", sys.stdout)
            return ""
            
        res = preds[0][-1]
        sorted_indices = torch.argsort(res, descending=True)
        for idx in sorted_indices:
            if idx not in ['xxbos', 'xxpad', 'xxunk', 'xxmaj']:
                if numericalize.vocab[idx].lower().startswith(final_word):
                    print("exiting complete word loop without error", sys.stdout)
                    return numericalize.vocab[idx][len(final_word):]
        print("exiting complete word loop without error without string", sys.stdout)
        return ""

In [9]:
print(complete_word(lstm_model, "this is", "th"))

<coroutine object complete_word at 0x7fd3851d21c0>


<ipython-input-9-a19ed4d897ed>:1: RuntimeWarning: coroutine 'complete_word' was never awaited
  print(complete_word(lstm_model, "this is", "th"))


In [13]:
def get_word():
    word = await complete_word(lstm_model, "this is", "th")
    return word

inside complete word loop <ipykernel.iostream.OutStream object at 0x7fd444d125e0>
exiting complete word loop without error <ipykernel.iostream.OutStream object at 0x7fd444d125e0>


In [8]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

In [52]:
from fastai.text.all import *

In [53]:
lstm_model = load_learner('../api/5epochs_imdb_lm.pkl')

In [54]:
lstm_model.dls.vocab

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 'the',
 '.',
 ',',
 'and',
 'a',
 'of',
 'to',
 'is',
 'it',
 'in',
 'i',
 'this',
 'that',
 '"',
 "'s",
 '-',
 'was',
 '\n\n',
 'as',
 'with',
 'for',
 'movie',
 'but',
 'film',
 'you',
 ')',
 'on',
 "n't",
 '(',
 'not',
 'are',
 'he',
 'his',
 'have',
 'be',
 'one',
 'all',
 'at',
 'they',
 'by',
 'an',
 'who',
 'from',
 'so',
 'like',
 '!',
 'there',
 'or',
 'just',
 'her',
 'do',
 'about',
 'has',
 'out',
 "'",
 'if',
 'what',
 'some',
 '?',
 'good',
 'when',
 'more',
 'very',
 'she',
 'up',
 'would',
 'no',
 '…',
 'time',
 'even',
 'my',
 'can',
 'their',
 'which',
 'only',
 'story',
 'really',
 'see',
 'had',
 'were',
 'did',
 'me',
 'well',
 'we',
 'does',
 'than',
 'much',
 ':',
 'could',
 'bad',
 'get',
 'been',
 'people',
 'other',
 'great',
 'into',
 'also',
 'will',
 'first',
 'because',
 'him',
 'how',
 'most',
 'them',
 'made',
 'make',
 'its',
 'then',
 'way',
 'too',
 'movies',
 '

In [5]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
neutral_model = AutoModelWithLMHead.from_pretrained("lvwerra/gpt2-imdb")
neutral_model.to('cuda')

/anaconda/envs/fastai2/lib/python3.8/site-packages/transformers/modeling_auto.py:821: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [60]:
vocab = sorted(lstm_model.dls.vocab)

In [61]:
vocab[:100]

['\t',
 '\n\n',
 '\n\n\t',
 '\n\n\x85',
 '\n\n▁',
 '\n▁',
 '!',
 '"',
 '")',
 '"a',
 '"the',
 '#',
 '$',
 '%',
 '&',
 "'",
 "''",
 "'bout",
 "'cause",
 "'cos",
 "'cuz",
 "'d",
 "'em",
 "'ll",
 "'m",
 "'nuff",
 "'re",
 "'s",
 "'ve",
 '(',
 '(:',
 '(=',
 ')',
 '):',
 '*',
 '+',
 '+1',
 ',',
 ',etc',
 '-',
 '--',
 '--a',
 '--all',
 '--and',
 '--as',
 '--even',
 '--i',
 '--or',
 '--polarisdib',
 '--shelly',
 '--the',
 '-.-',
 '-1',
 '-10',
 '-100',
 '-2',
 '-20',
 '-4',
 '-5',
 '-50',
 '-7',
 '-8',
 '-a',
 '-all',
 '-an',
 '-and',
 '-ap3-',
 '-apparently',
 '-as',
 '-at',
 '-bad',
 '-because',
 '-but',
 '-celluloid',
 '-chris',
 '-d',
 '-facts',
 '-for',
 '-good',
 '-great',
 '-he',
 '-his',
 '-how',
 '-i',
 '-if',
 '-in',
 '-including',
 '-is',
 '-it',
 '-john',
 '-just',
 '-la',
 '-ld',
 '-like',
 '-maybe',
 '-more',
 '-mostly',
 '-no',
 '-not',
 '-now']

In [62]:
vocab.startswith('li')

AttributeError: 'list' object has no attribute 'startswith'

In [81]:
def complete_word_transformer(language_model, tokenizer, vocab, text, final_word):
    if len(text) == 0:
        return ''
    ids = tokenizer.encode(text)
    t = torch.LongTensor(ids)[None].to('cuda')
    logits = language_model.forward(t)[0][-1][-1]
    sorted_indices = torch.argsort(logits, descending=True)
    for tk_idx in sorted_indices:
        word = tokenizer.decode([tk_idx.cpu()])
        if word.lower().startswith(final_word):
            if word.lower() != final_word and word.lower() in vocab:
                return word[len(final_word):]
    return ""

In [82]:
complete_word_transformer(neutral_model, tokenizer, vocab, "the real", "iss")

'ue'

In [73]:
text = 'hello'

In [76]:
text.startswith??

In [ ]:
'abc'.startwith